# Bellman-Ford Algorithm

In an earlier section on Dijkstra's algorithm, we saw an algorithm to compute the shortest path between a source and destination in a graph. However, the Dikstra's algorithm requires the complete information of the graph to compute the shortest path. Thus on a computer network, if a node has to compute the shortest path with respect to itself, it will need the entire network information. Is there a way to compute the shortest path without having the complete view of the graph? Can we compute the shortest path in a distributed manner, where nodes help each other to figure out the shorted path? In this notebook, we introduce such an algorithm that can be executed in a distributed manner.

In this notebook we introduce the Bellman-Ford algorithm that computes the shortest path from a source node to all remaining nodes in the graph. In practice, the Bellman-Ford algorithm is run in a distributed setting, where nodes update their distance metrics, and share their computed metrics with their neighbours. On reception of an updated metric from a neighbour, nodes update their own metrics, and then shares their updated metrics with their neighbours.


In the context of routing, [Distance-Vector Routing algorithms](https://en.wikipedia.org/wiki/Distance-vector_routing_protocol) like the [Routing Information Protocol](https://en.wikipedia.org/wiki/Routing_Information_Protocol) uses the Bellman-Ford algorithm to compute the routing metrics. 

### Table of Contents
1. [Building the graph](#graph1)
2. [About the algorithm](#algorithm)
3. [Implementation on undirected, weighted graph](#implementation)
4. [Detecting Negative-weight cycles](#negativeCycles)
5. [Computing shortest path on graphs with negative edges](#comparison)
6. [Proof of Correctness](#proof)
7. [References](#references)

## Fetching required modules

Before, we get started lets import code modules which would make us focus on understanding the Bellman-Ford algorithm.

In [ ]:
import os, sys, subprocess
from os.path import dirname, join, abspath
import warnings
warnings.filterwarnings('ignore')

if 'google.colab' in str(get_ipython()):
  print('Running on CoLab')
  !git clone https://github.com/cni-iisc/networks-course.git
  !ln -sf networks-course/modules modules
else:
  print('Not running on CoLab; please make sure to install the packages listed in requirements.txt')
    
sys.path.insert(0, abspath(join(dirname("modules"), '..')))  
from modules.create_graph import *
from modules.visualize_graph import *

<a class="anchor" id="graph1"></a>
## Building the graph

To establish the implementation of the Bellman-Ford's algorithm, we create a weighted, undirected graph which is the same graph used in the [previous notebook](./1_dijkstra.ipynb) for Dijkstra's algorithm.

In [ ]:
a = Node()
b = Node()
c = Node()
d = Node()
e = Node()
f = Node()
g = Node()
graph = Graph.createGraph([a, b, c, d, e, f], directed=False)


graph.add_Edge(a,b,1)
graph.add_Edge(a,c,10)
graph.add_Edge(a,e,2)
graph.add_Edge(b,c,10)
graph.add_Edge(b,d,6)
graph.add_Edge(c,d,1)
graph.add_Edge(c,f,10)
graph.add_Edge(d,e,3)

visualizeGraph(graph, "bellmanFord")

<a class="anchor" id="algorithm"></a>
## About the algorithm

### Psuedo code
<pre>1 <b>function</b> BellmanFord(<i>list</i> graph, <i>node</i> source)
2   <b>for each</b> vertex v <b>in</b> graph:
3       distance[v]&nbsp;:= <b>INFINITY</b>             
4       predecessor[v]&nbsp;:= <b>null</b>         
5   
6   distance[source]&nbsp;:= 0              
7 
8
9   <b>for</b> i <b>from</b> 1 <b>to</b> size(vertices)-1: //just |V|-1 repetitions; i is never referenced
10       <b>for each</b> edge (u, v) <b>with</b> weight w <b>in</b> edges:
11           <b>if</b> distance[u] + w &lt; distance[v]:
12               distance[v]&nbsp;:= distance[u] + w
13               predecessor[v]&nbsp;:= u
14               
15   <b>for each</b> edge (u, v) <b>with</b> weight w <b>in</b> edges:
16       <b>if</b> distance[u] + w &lt; distance[v]:
17           <b>error</b> "Graph contains a negative-weight cycle"
18           <b>break</b>
19
20   <b>return</b> distance[], predecessor[]
</pre>

<a class="anchor" id="implementation"></a>
## Implementing Bellman-Ford algorithm

In [ ]:
def bellmanFord(graph, sourceNode, targetNode):
    sourceNodeIndex = graph.get_nodeIndex(sourceNode)
    targetNodeIndex = graph.get_nodeIndex(targetNode)
    nodeList = graph.get_allNodes()
    
    # Make an array keeping track of distance from node to any node
    # in self.nodes. Initialize to infinity for all nodes but the 
    # starting node, keep track of "path" which relates to distance.
    # Index 0 = distance, index 1 = previous_hops
    dist = [None] * len(nodeList)
    previous = [None] * len(nodeList)
    for i in range(len(dist)):
        dist[i] = float("inf")
        previous[i] = []
        
    dist[sourceNodeIndex] = 0

    
    # Step 2: Relax all edges |V| - 1 times. A simple shortest  
    # path from src to any other vertex can have at-most |V| - 1  
    # edges 
    
    for i in range(len(nodeList) - 1):
        # update dist value and parent index of adjacent values of picked vertex.
        # consider those which are still in queue.
        for u, v, w in graph.get_allEdges():
            #print("Itertion : ",i, ".u = ",u, "v = ",v, "w = ",w, " dist[u] = ", dist[u], "dist[v] = ",dist[v], "w = ",w)
            
            if dist[u]!= float("Inf") and dist[u] + w < dist[v]:
                dist[v] = dist[u] + w
                
                previous[v] = list(previous[u])
                previous[v].append(u)      
                #print("Itertion : ",i,". Distance updated. Node = ", v, ". Distance = ",dist[v], "Previous = ",  previous[v])
            if (graph.directed == False):
                temp = u
                u = v
                v = temp
                if dist[u]!= float("Inf") and dist[u] + w < dist[v]:
                    dist[v] = dist[u] + w

                    previous[v] = list(previous[u])
                    previous[v].append(u)      
                    #print("Itertion : ",i,". Distance updated. Node = ", v, ". Distance = ",dist[v], "Previous = ",  previous[v])
                
    # check for negative weight cycles. If path obtained from above step (shortest distances)
    # is shorter, there's a cycle. So quit.
    for u, v, w in graph.get_allEdges():
        if dist[u] != float("Inf") and dist[u] + w < dist[v]:
            print("Negative Cycles Detected at %d" %v)
            return dist[targetNodeIndex], ["Negative Cycle Detected"]
        
    #before returning add the targetNodeIndex to visited Nodes
    previous[targetNodeIndex].append(targetNodeIndex)
    return dist[targetNodeIndex], previous[targetNodeIndex]

In [ ]:
sourceNode = a
targetNode = f

distances, previousNodes = bellmanFord(graph, sourceNode, targetNode)

path = None
if distances != float("inf"):
    print("Total distance to sourceNode %d from targetNode %d is %d" % (graph.get_nodeIndex(sourceNode),  graph.get_nodeIndex(targetNode), distances))
    edgelist = []
    for i in range(len(previousNodes)):
                try:
                    wt = graph.get_edgeWeight(previousNodes[i],previousNodes[i+1])
                    edgelist.append((previousNodes[i],previousNodes[i+1], wt))

                except IndexError:
                    break

    path = displayPath(edgelist, "bellmanFord")
else:
    print("Total distance from sourceNode %d to targetNode %d is %f" % (graph.get_nodeIndex(sourceNode),  graph.get_nodeIndex(targetNode), distances))

path

<a class="anchor" id="comparison"></a>
## Computing the shortest path in a graph with negative edge weights

Now, we are interested to compare the Dijsktra's shortest path algorithm with the Bellman Ford algorithm to find the difference between the two algorithms. For nodes which have non-negative edges, we find Dijkstra to be a faster and more efficient solution. However, when there are some edges that have negative weights, Dijkstra's algorithm is not able to correctly compute the shortest path. This is due to the greedy-decision making condition of selecting the node with the minimum distance. Bellman-Ford on the other hand is able to handle the negative weights better.

In [ ]:
def dijkstra(graph, sourceNode, targetNode):
    sourceNodeIndex = graph.get_nodeIndex(sourceNode)
    targetNodeIndex = graph.get_nodeIndex(targetNode)
    nodeList = graph.get_allNodes()

    # Make an array keeping track of distance from node to any node
    # in self.nodes. Initialize to infinity for all nodes but the 
    # starting node, keep track of "path" which relates to distance.
    # Index 0 = distance, index 1 = previous_hops
    dist = [None] * len(nodeList)
    previous = [None] * len(nodeList)
    for i in range(len(dist)):
        dist[i] = float("inf")
        previous[i] = [sourceNodeIndex]

    
    dist[sourceNodeIndex] = 0
 
    # Queue of all nodes in the graph
    # Note the integers in the queue correspond to indices of node
    # locations in the self.nodes array
    queue = [i for i in range(len(nodeList))]
    
    # Set of numbers seen so far
    seen = set()
    while len(queue) > 0:
        # Get node in queue that has not yet been seen
        # that has smallest distance to starting node
        min_dist = float("inf")
        min_node = None
        for n in queue: 
            if dist[n] < min_dist and n not in seen:
                min_dist = dist[n]
                min_node = n
        try:
            # Add min distance node to seen, remove from queue
            queue.remove(min_node)
            seen.add(min_node)


            # Get all next hops -> all shortest paths
            connections = graph.get_connections(min_node)

            # For each connection, update its path and total distance from 
            # starting node if the total distance is less than the current distance
            # in dist array

            for (node, weight) in connections.items(): 

                tot_dist = weight + min_dist

                if tot_dist < dist[node]:
                    #if less, print the update
                    dist[node] = tot_dist
                    previous[node] = list(previous[min_node])
                    previous[node].append(node)
                    
                    if node != targetNodeIndex :
                        continue
                    else:
                        return dist[node], previous[node]
        except ValueError:
                print ("Distance to node %d is %d" %(targetNodeIndex, sys.maxsize))
                return None, None

In [ ]:
a = Node()
b = Node()
c = Node()
d = Node()
e = Node()
f = Node()

compare_graph = Graph.createGraph([a, b, c, d, e, f], directed=True)



compare_graph.add_Edge(a,b,5)
compare_graph.add_Edge(b,c,10)
compare_graph.add_Edge(b,d,5)
compare_graph.add_Edge(c,e,-20)
compare_graph.add_Edge(e,d,10)
compare_graph.add_Edge(d,f,5)


visualizeGraph(compare_graph, "bellmanFord_compare")
visualizeGraph(compare_graph, "Dijkstra_compare")

adjacentMatrix = compare_graph.get_adjMatrix()

sourceNode = a
targetNode = f

distances_Dijks, previousNodes_Dijks = dijkstra(compare_graph, sourceNode, targetNode)
distances_BF, previousNodes_BF = bellmanFord(compare_graph, sourceNode, targetNode)


print ("\t\t  | Dijkstra\t| Bellman-Ford")
print ("Distance Computed | %d\t\t| %d" % (distances_Dijks, distances_BF))
print ("Previous Nodes    | %s| %s" % (previousNodes_Dijks, previousNodes_BF))

edgelistBF = []
for i in range(len(previousNodes_BF)):
            try:
                wt = graph.get_edgeWeight(previousNodes_BF[i],previousNodes_BF[i+1])
                edgelistBF.append((previousNodes_BF[i],previousNodes_BF[i+1], wt))

            except IndexError:
                break
edgelistDij = []
for i in range(len(previousNodes_Dijks)):
            try:
                wt = graph.get_edgeWeight(previousNodes_Dijks[i],previousNodes_Dijks[i+1])
                edgelistDij.append((previousNodes_Dijks[i],previousNodes_Dijks[i+1], wt))

            except IndexError:
                break
                

In [ ]:
displayPath(edgelistBF, "bellmanFord_compare")

In [ ]:
displayPath(edgelistDij, "Dijkstra_compare")

<a class="anchor" id="negativeCycles"></a>
## Detecting Negative Weights Cycles

A cycle is considered a negative weight cycle when the sum of all its edges are negative. If a negative cycle exisits in the graph that is reachable from the source node, then an additional walk over the negative cycle is required to get the shortest path. Bellman-Ford algorithm can detect negative weight cycles which makes it useful to determine the presence of a shortest path in the graph.



In [ ]:
a = Node()
b = Node()
c = Node()
d = Node()
e = Node()
f = Node()

negative_wt_graphs = Graph.createGraph([a, b, c, d, e, f], directed=True)



negative_wt_graphs.add_Edge(a,b,5)
negative_wt_graphs.add_Edge(b,c,10)
negative_wt_graphs.add_Edge(b,d,5)
negative_wt_graphs.add_Edge(c,e,-20)
negative_wt_graphs.add_Edge(e,d,10)
negative_wt_graphs.add_Edge(d,f,5)
negative_wt_graphs.add_Edge(e,b,-5)





sourceNode = a
targetNode = f

bellmanFord(negative_wt_graphs, sourceNode, targetNode)
visualizeGraph(negative_wt_graphs, "neg_bellmanFord")

<a class="anchor" id="proof"></a>
## Proof of correctness

Let $G(V,E)$ be a weighted, directed graph where $V$ represents the set of all nodes and $E$ represents the set of all edges in the graph. Let $d_{k}[u]$ represent the distance computed by the algorithm after $k$ iterations, and $\delta(u)$ represent the actual shortest distance existing from the source node $s$ to the target node $u$.

*To Prove:* For a graph $G(V,E)$ with no negative weight cycles, $d_{|V|-1}[u] = \delta(u)\; \forall u \in V$.

Let us consider that node $t$ on the graph is reachable from the source node $s$, through a path $p$ such that $p = \{v_{0}, v_{1},....., v_{K}\}$ where $v_{0} = s$ and  $v_{K} = t$. Let path $p$ be the shortest path from the source node $s$ to node $t$ which means at most it has $|V| - 1$ edges as part of the path, thus, $K \leq |V| - 1$.

*Claim :* For node $v_{l+1}$ in $p$, $\delta(v_{l+1}) = \delta(v_{l})+len(v_{l}, v_{l+1})$.
If not, there exists another shorter path to $v_{l+1}$, and replacing the section of $p$ till $v_{l+1}$ with this shorter distance path will yield a total smaller distance to $t$. This contradicts the definition of $p$.

We will now prove the correctness of the Bellman-Ford algorithm via induction.

Induction hypothesis: If at iteration $k$, $d_{k}[v_{k}] = \delta(v_{k})$. Then after $k+1$ iterations, $d_{k+1}[v_{k+1}] = \delta(v_{k+1})$.

Proof:
At iteration $k+1$,
\begin{align*}
 d_{k+1}[v_{k+1}] &= \min\{d_{k}[v_{k+1}], \min_{u}(d_{k}[u]+len(u,v_{k+1}))\}\\
 & \le d_{k}[v_{k}] + len(v_{k}, v_{k+1})\\
 &= \delta(v_{k}) + len(v_{k}, v_{k+1})\\
 & = \delta(v_{k+1}),
\end{align*}
where the first inequality follows as we are considering a particular term among all terms in the minimisation, the second equality follows from the induction hypothesis about $d_{k}[v_{k}]$, and the third equality follows from the claim above about $\delta(v_{k+1})$.

If we start the algorithm such that $d_{0}[s] = 0$ and $d_{0}[v] = \infty$, for all $v \in V \setminus s$, the induction hypothesis is satisfied. Hence, the correctness of the Bellman-Ford algorithm is proved.

<a class="anchor" id="references"></a>
## Reference

- [1] Leiserson, Charles Eric, Ronald L. Rivest, Thomas H. Cormen, and Clifford Stein. Introduction to algorithms. Chapter 24. Vol. 6. Cambridge, MA: MIT press, 2001.
- [2] Bellman-Ford Algorithm on [Wikipedia](https://en.wikipedia.org/wiki/Bellman%E2%80%93Ford_algorithm)
- [3] Implementation of Bellman-Ford inspired form [Geeks for Geeks](https://www.geeksforgeeks.org/bellman-ford-algorithm-dp-23/)